# Information Retrieval and Web Analytics

# Part 3: Ranking


In [1]:
# mount google drive if using google collab, else skip
# we are not using it because it is more comfortable to use jupyter lab

BASEDIR = '.'

try:
    from google.colab import drive
    drive.mount('/content/drive')
    BASEDIR = 'drive/MyDrive'
    
except ModuleNotFoundError:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# required imports for the notebook

import json
import csv
import math
import numpy as np
from array import array
from collections import defaultdict, Counter
import functools
from sklearn.manifold import TSNE


from nltk.stem import PorterStemmer

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
import pandas as pd
import rank_bm25


# queries for testing
queries = [
    "keep us posted",
    "ian update",
    "disney world",
    "climate change",
    "hit state"
]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# open results from last practice
tweets = pd.read_csv(f'{BASEDIR}/data/processed_tweets.csv')
tweets = tweets.reset_index()  # make sure indexes pair with number of rows

### Content from practice part 2

In [4]:
import time

RED = "\033[91m"
WHITE = "\033[0m"


def benchmark(func):
    """
    Decorador que te mide el tiempo que tarda la funcion en ejecutarse.
    Se puede usar como cualquier funcion, e.g. benchmark(func),
    pero al ser un decorador la gracia que tiene es que al hacer
    @benchmark
    def func():...
    cada vez que uses func() estaras usando benchmark(func)()
    :param func: la funcion que quieres testear
    :return: la funcion original envuelta por el codigo de testeo
    """
    def inner(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Time taken for {RED}{func.__name__}{WHITE}: {end - start:.4f}')
        return result

    return inner

def remove_punctuation(text):

    """
    Removes the characters:
    !\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789
    from the text.
    """

    chars_to_remove = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789"

    tr = str.maketrans("", "", chars_to_remove)

    return text.translate(tr)

In [5]:
# reuse of the function shown in class to transform text into lowercase and erase stop words in queries
def build_terms(line):
    """
    Preprocess the line removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    line = str(line).lower()
    
    # tremendo pero aligual rompe algo despues
    line = remove_punctuation(line)
    
    line = line.split()  # Tokenize the text to get a list of terms
    line = [x for x in line if x not in stop_words]  # eliminate the stopwords
    line = [stemmer.stem(word) for word in line] # perform stemming (HINT: use List Comprehension)
    return line


@benchmark
def create_index(tweets):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    title_index = {}  # dictionary to map page titles to page ids

    for tweet in tweets.itertuples(index=True):  # Remember, lines contain all documents from file
        tweet_text = tweet.full_text
        
        # tweet id
        tweet_id = int(tweet.id.split("_")[1])

        terms = str(tweet_text).split(" ")  # page_title + page_text

        title_index[tweet_id] = tweet.user  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is "web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                current_page_index[term][1].append(position)
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term] = [tweet_id, array('I', [position])]  #'I' indicates unsigned int (int in Python)

        # merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
    return index, title_index


In [6]:
index, title_index= create_index(tweets)

Time taken for create_index: 0.2927


### 5 Text queries

In [7]:
# def query(text)?:
# build terms(query)

# index = tremendo index
# foreach term in query
# index = index[term in entry] <- boolean mask, conjunctive AND

# return index <- the doc ids
# or
# return tweets[original_text][index] <- get original text of tweets containing all elements in query

def query(text, tweet_index=""):
    """
    search for a given text in the tweet collection using the
    inverted index we previously computed
    :param text: the query text
    :param tweet_index: inverted index of the collection, named as such because context of practice
    :return: list of tweet ids containing all (treated) terms in the query
    """
    
    # necessary step since same treatment applied to tweets
    terms = build_terms(text)
    
    # select tweet index, defaults to global index but can be specified
    tweet_index = tweet_index if tweet_index else index
    
    plausible_ids = []
    for query_term in terms:
        # tweet_index[query_term] is list of tweet ids containing query term + position(s) in text, could be useful in the future
        # plausible_ids[query_term] = tweet_index[query_term]
        
        # using sets is convenient for using reduce
        plausible_ids.append(set(term_pos[0] for term_pos in tweet_index[query_term]))
        
    # reduce list of sets to intersection of all
    relevant_ids = functools.reduce(lambda a, b: a.intersection(b), plausible_ids) if plausible_ids else []
                             
    return relevant_ids
        

### Ranking results: TF-IDF + cosine score

In [8]:
# rank(query):
# query = build_terms(query)

# scores = {}
# length = {}
# foreach term in query:
# w_q = TF-IDF(term, query), docids = query(term)
# foreach docid in docids:
# scores[docid] += TF-IDF(term, tweets.full_text[docid]) * w_q
# ::
# foreach docid:
# scores[d] /= len(tweets.full_text[docid])

# sort scores, return top K


# relevant documents = query(query)
# foreach document in relevant_documents
# TF-IDF(document, query)



# TF-IDF(document, query):

# len(query(term)) is df(term) if term is one word 


def tf_idf(term_freq, document_freq, collection_len):
    if term_freq == 0 or document_freq == 0:
        return 0
    return (1 + math.log(term_freq)) * math.log(collection_len/document_freq)


def doc_score(doc_id, collection_index=index, collection=""):
    """
    vector de scores para el documento dado, es lo que hay que usar para
    la document length
    
    tremendo usarlo como {doc_id: doc_score(doc_id)} para todos los ids
    :param doc_id: document id que mirar
    :params: se supone que así será más flexible pero los defaults van finos asi que na
    :return: diccionario de terms y pesos, util para normalizar documentos
    """
    result={}
    
    collection = collection if collection else {tweet.id: tweet.full_text for tweet in tweets.itertuples(index=True)}
    collection_len = len(collection)
    
    document = str(collection[doc_id]).split(" ")
    term_frequencies = Counter(document)
    
    for term in document:
        document_freq = len(query(term, tweet_index=collection_index))
        result[term] = tf_idf(term_frequencies[term], document_freq, collection_len)
    return result


@benchmark
def collection_vectors(collection="", collection_index=index):
    """
    multi diccionario de documentos, terms y sus valores tf-idf
    """
    document_vectors = {}
    
    collection = collection if collection else {tweet.id: tweet.full_text for tweet in tweets.itertuples(index=True)}
    for doc_id, document in collection.items():
        document_vectors[doc_id] = doc_score(doc_id, collection_index=collection_index, collection=collection)
        
    return document_vectors


In [9]:
document_lengths = collection_vectors()

Time taken for collection_vectors: 23.6884


In [10]:
def cosine_score(query_text, collection_index=index, collection="", lengths=document_lengths, k=10):
    """
    computes cosine score of all documents in a collection against a query and ranks them
    accordingly
    """

    collection = collection if collection else {tweet.id: tweet.full_text for tweet in tweets.itertuples(index=True)}
    collection_len = len(collection)
           
    scores = {doc_id: 0 for doc_id in collection.keys()}
    
    # esto seguramente este mal
    # length = {doc_id: len(str(document).split(" ")) for doc_id, document in collection.items()}
    
    query_terms = build_terms(query_text) # necessary step since same treatment applied to tweets
    
    # dictionary of frequency of each term in the query
    query_frequencies = Counter(query_terms)
    
    for term in query_terms:
        # query of a term returns the set of documents containing the term
        document_freq = len(query(term, tweet_index=collection_index))
        
        query_weight = tf_idf(query_frequencies[term], document_freq, collection_len)
        
        """
        for term in query_terms:
        
        # query of a term returns the set of documents containing the term
        document_freq = len(query(term, tweet_index=collection_index))
        
        query_weight = tf_idf(query_frequencies[term], document_freq, collection_len)
        
        # hasta aqui esta bien probablemente, despues pasa algo raro
        for doc_id, document in collection.items():
            # counter of distinct terms in document
            term_frequencies = Counter(str(document).split(" "))
            document_weight = tf_idf(term_frequencies[term], document_freq, collection_len)
            scores[doc_id] += query_weight * document_weight
        """
            
        for doc_id, document in collection.items():

            term_frequencies = Counter(str(document).split(" "))
            document_weight = tf_idf(term_frequencies[term], document_freq, collection_len)

            doc_vec = list(lengths[doc_id].values())
            scores[doc_id] = query_weight * document_weight
            
    scores = {doc_id: score/np.linalg.norm(list(lengths[doc_id].values())) for doc_id, score in scores.items()}
        
    # if k is 0 return whole doc id list
    k = k if k else collection_len
    
    doc_ids_sorted = sorted(scores, key=scores.get, reverse=True)[:k]
    return {doc_id: scores[doc_id] for doc_id in doc_ids_sorted}

In [12]:
for query in queries:
  cosine_scores = {cosine_id: score for cosine_id, score in cosine_score(query).items()}
  print(query)
  print(cosine_scores)

TypeError: ignored

### Ranking results: our score + cosine similarity 

In [13]:
def our_score(query_text, collection_index=index, lengths=document_lengths, k=10):
    """
    uhh multiply tf idf score by log popularity of tweet so our ranking is sensitive to tweet popularity
    """
    tweet_data = [(tweet.id, tweet.full_text, tweet.retweet_count + tweet.favorite_count + 1) for tweet in tweets.itertuples(index=True)]
    
    collection = {tweet_id: text for tweet_id, text, _ in tweet_data}
    
    score_factor = {tweet_id: np.log10(pop_value) for tweet_id, _, pop_value in tweet_data}
    
    # tf-idf and cosine score
    base_score = cosine_score(query_text, collection_index=index, collection=collection, lengths=document_lengths, k=0)

    # get new scores by multiplying tf idf by tweet popularity
    scores = {tweet_id: score * score_factor[tweet_id] for tweet_id, score in base_score.items()}
    tweet_ids_sorted = sorted(scores, key=scores.get, reverse=True)[:k]
    return {tweet_id: scores[tweet_id] for tweet_id in tweet_ids_sorted}

In [14]:
for query in queries:
  custom_scores = {custom_id: score for custom_id, score in our_score(query).items()}
  print(query)
  print(custom_scores)

TypeError: ignored

### Ranking results: BM25

In [15]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

bm_collection = {tweet.id: build_terms(tweet.full_text) for tweet in tweets.itertuples(index=True)}
bm_docs = bm_collection.values()
#bm_collection = [build_terms(tweet.full_text) for tweet in tweets.itertuples(index=True)]

bm25 = BM25Okapi(bm_docs)

In [17]:
for query in queries:
  print("\n"+query)
  retrieved_docs = bm25.get_top_n(build_terms(query), list(bm_collection.values()), n=20)

  bm_doc_ids = bm_collection.keys()

  # bm uses int indexes so we must return them to normal here
  bm_doc_retrieved_ids = [list(bm_doc_ids)[list(bm_docs).index(doc)] for doc in retrieved_docs]

  for doc in retrieved_docs:
      # print(f"{RED}{list(bm_doc_ids)[list(bm_docs).index(doc)]}:{WHITE}\t{doc}")
      print(list(bm_doc_ids)[list(bm_docs).index(doc)])


keep us posted
doc_3377
doc_1
doc_433
doc_2160
doc_1069
doc_3955
doc_2045
doc_866
doc_1873
doc_2384
doc_1546
doc_2677
doc_1320
doc_3996
doc_2495
doc_2640
doc_2330
doc_2925
doc_1187
doc_3700

ian update
doc_1933
doc_117
doc_117
doc_117
doc_117
doc_117
doc_2316
doc_2340
doc_2249
doc_1771
doc_1907
doc_344
doc_1830
doc_1830
doc_1244
doc_1244
doc_3242
doc_2105
doc_460
doc_1861

disney world
doc_3875
doc_3029
doc_3850
doc_808
doc_3397
doc_3424
doc_1614
doc_2833
doc_3795
doc_889
doc_2643
doc_420
doc_1222
doc_1567
doc_2821
doc_804
doc_3076
doc_675
doc_2218
doc_3407

climate change
doc_3668
doc_511
doc_1958
doc_3945
doc_147
doc_889
doc_3745
doc_1682
doc_3661
doc_220
doc_1527
doc_3469
doc_238
doc_2506
doc_925
doc_3162
doc_2845
doc_802
doc_2497
doc_3622

hit state
doc_481
doc_842
doc_45
doc_2947
doc_1973
doc_1525
doc_2692
doc_665
doc_1918
doc_1695
doc_2155
doc_1482
doc_1482
doc_1258
doc_924
doc_3635
doc_2001
doc_988
doc_231
doc_2035


### Top-20 list of documents

In [18]:
top_20_bm = tweets[tweets["id"].isin(bm_doc_retrieved_ids)]
top_20_custom = tweets[tweets["id"].isin(custom_scores.keys())]
top_20_cosine = tweets[tweets["id"].isin(cosine_scores.keys())]

NameError: ignored

In [19]:
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def word2vec(tweets):
    clean_tweets = []
    for tweet in tweets.itertuples(index=True): 
            clean_tweets.append(str(tweet.full_text))
    model = Word2Vec(clean_tweets, workers=4, min_count=50, window=10, sample=1e-3)

    X = model.wv[model.wv.key_to_index]

    tsne = TSNE(n_components=2, perplexity=1)
    X_tsne = tsne.fit_transform(X)

    plt.scatter(X_tsne[:, 0], X_tsne[:, 1])

    plt.show()
word2vec(top_20_bm)
word2vec(top_20_custom)
word2vec(top_20_cosine)

AttributeError: ignored

In [ ]:
top_docs_per_query = {query: cosine_score(query, k=20) for query in queries}

In [ ]:
top_docs_per_query